In [1]:
import pandas as pd
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
import torch
from common import get_instagram_sessions, get_text_embedding, get_text_embeddings_splitted
from collections import  defaultdict
from scipy.spatial.distance import cosine, euclidean
import plotly.express as px
model_id = "stabilityai/stable-diffusion-2"

device = 'cuda' if torch.cuda.is_available() else 'cpu'


def get_word_lists():
    with open('apologetic.txt', 'r') as apologetic_file:
        apologetic_context = apologetic_file.readlines()
    with open('comforting.txt', 'r') as comfort_file:
        comfort_context = comfort_file.readlines()
    with open('stopping.txt', 'r') as stopping_file:
        stopping_context = stopping_file.readlines()

    return apologetic_context, comfort_context, stopping_context


2022-12-12 03:33:40.840420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 03:33:40.955511: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-12 03:33:41.572216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-12 03:33:41.572283: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
sessions = get_instagram_sessions()
all_comments = [y['content'] for x in sessions for y in x['comments']]
# Use the Euler scheduler here instead
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, revision="fp16", torch_dtype=torch.float16, max_len=300, truncation=True, padding=True)
# pipe.tokenizer.model_max_length = 300
# pipe = pipe.to("cuda")

apol, comf, stop = get_word_lists()
apol, comf, stop = [[x.strip() for x in y] for y in [apol, comf, stop]]

# pipe._encode_prompt(apol, device, 1, True, None)
# Get the encodings for all the words
architype_encodings = []
for l in [apol, comf, stop]:
    m_l = get_text_embedding(l)
    architype_encodings.append(m_l)




Doing instagram_100_sessions/Compiled and Totaled Sessions 2020/CB IG Compiled Coding Set 1 - 2020 (Revised 2.0).xlsx
Doing sheet 1 Overall CB - Yes
Doing sheet 2 Overall CB - No
Doing sheet 3 Overall CB - No
Doing sheet 4 Overall CB - No
Doing sheet 5 Overall CB - No
cannot convert float NaN to integer
louiscabb
(nan, nan, nan)
So would it take a additional 7days to arrive or how much? 
nan
Doing sheet 6 Overall CB - No
Doing sheet 7 Overall CB - No
cannot convert float NaN to integer
gianna_devincenzo
(nan, nan, nan)
@_brittany_keyes_ 
nan
Doing sheet 8 Overall CB - Yes
Doing sheet 9 Overall CB - No
Doing sheet 10 Overall CB - No
Doing sheet 11 Overall CB - No
Doing sheet 12 Overall CB - No
Doing sheet 13 Overall CB - Yes
Doing sheet 14 Overall CB - No
Doing sheet 15 Overall CB - No
Doing sheet 16 Overall CB - No
'float' object has no attribute 'strip'
elyse_turney
(nan, nan, nan)
nan
0
Doing sheet 17 Overall CB - Yes
Doing sheet 18 Overall CB - Yes
Doing sheet 19 Overall CB - No
Doi

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

In [3]:
# Encode the instagram session texts
comment_encodings = get_text_embeddings_splitted(all_comments, 256)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifie

In [4]:
# Compare the encoded comments with the architypes
comment_stats = defaultdict(list)
for i_c, ce in enumerate(comment_encodings):
    for i, arch in enumerate(architype_encodings):
        min_cos = min_euc = float('inf')
        mean_cos = mean_euc = 0
        for arch_i in arch:
            cos_dist = cosine(arch_i, ce)
            euc_dist = euclidean(arch_i, ce)
            min_cos = cos_dist if cos_dist<min_cos else min_cos
            min_euc = euc_dist if euc_dist<min_euc else min_euc
            mean_cos += cos_dist
            mean_euc += euc_dist
        mean_euc, mean_cos = mean_euc/len(arch), mean_cos/len(arch)
        comment_stats[i_c].append({
            'arch': i,
            'min_cos': min_cos,
            'min_euc': min_euc,
            'mean_cos': mean_cos,
            'mean_euc': mean_euc
        })

comment_data = []
for i, c in enumerate(all_comments):
    comment_datum = {}
    for v in comment_stats[i]:
        arch = v['arch']
        for k1, v1 in v.items():
            if k1 == 'arch':
                continue
            comment_datum[f'{k1}_{arch}'] = v1
    comment_datum['comment'] = c
    comment_data.append(comment_datum)

In [5]:
df_comments = pd.DataFrame(comment_data)

In [6]:
df_comments

,min_cos_0,min_euc_0,mean_cos_0,mean_euc_0,min_cos_1,min_euc_1,mean_cos_1,mean_euc_1,min_cos_2,min_euc_2,mean_cos_2,mean_euc_2,comment
0,0.008197,8.111126,0.015723,11.060664,0.006484,7.213890,0.015913,11.019971,0.008038,8.032723,0.015697,10.946465,Zaaaayn
1,0.012835,10.149008,0.020010,12.557677,0.011332,9.536220,0.020214,12.545956,0.012996,10.213217,0.020110,12.514229,Larry
2,0.008699,8.355655,0.016143,11.217775,0.006895,7.438455,0.016310,11.169682,0.008333,8.178728,0.015937,11.040151,Niall_Å½Ã›_
3,0.030127,15.537373,0.038980,17.618789,0.030246,15.569288,0.041280,18.097596,0.022554,13.441738,0.030731,15.621138,How do u get a gif? I cant aave them to my phone
4,0.056284,21.233501,0.072391,24.053590,0.063283,22.517595,0.074888,24.463892,0.043945,18.759750,0.060874,22.001907,"Larry, Zayn being sexy, and Niall and Liam doi..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8448,0.007982,8.004100,0.015502,10.974006,0.006066,6.977427,0.015536,10.874890,0.007807,7.916058,0.015386,10.827153,"@spartacusfan7125 im still waitin on your ""goo..."
8449,0.052711,20.550373,0.069323,23.539519,0.060012,21.930006,0.071812,23.954413,0.037453,17.320288,0.057821,21.422588,Haha right!! @thenameisandy
8450,0.011190,9.472099,0.018511,12.027801,0.009045,8.515697,0.018882,12.050409,0.008673,8.337419,0.016051,11.176107,@jovanadragicevic ___¢___¢___¢___¢...
8451,0.036195,17.037941,0.042023,18.333767,0.034464,16.624519,0.042128,18.341152,0.035824,16.946238,0.041034,18.088460,___¢___¢___¢___¢ ja @ichliebejordb...


In [9]:
fig = px.scatter_3d(df_comments, x='min_cos_0', y='min_cos_1', z='min_cos_2', hover_data=['comment'])

In [11]:
fig.write_html('min_cos.html')

In [12]:
fig = px.scatter_3d(df_comments, x='min_euc_0', y='min_euc_1', z='min_euc_2', hover_data=['comment'])
fig.write_html('min_euc.html')

In [13]:
fig = px.scatter_3d(df_comments, x='mean_cos_0', y='mean_cos_1', z='mean_cos_2', hover_data=['comment'])
fig.write_html('mean_cos.html')

In [14]:
fig = px.scatter_3d(df_comments, x='mean_euc_0', y='mean_euc_1', z='mean_euc_2', hover_data=['comment'])
fig.write_html('mean_euc.html')